# Day 18: Operation Order

[*Advent of Code 2020 day 18*](https://adventofcode.com/2020/day/18) and [*solution megathread*](https://redd.it/kfeldk)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/18/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F18%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
testdata = """1 + 2 * 3 + 4 * 5 + 6
2 * 3 + (4 * 5)
5 + (8 * 3 + 9 + 3 * 4 * 3)
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2""".splitlines()

inputdata = downloaded['input'].splitlines()

In [6]:
def shunting_yard(expr_str, debug=False, part2=False):
    output = []
    operators = []
    # while there are tokens to be read:
    for token in expr_str.split():
        if debug:
            print(f'read a token: {token}')
        if token in {'+', '*'}:
            if debug:
                print(f'if token is an operator then: {token}')
            while (len(operators) > 0 and operators[-1] != '('
                   and (not part2 or
                        (not token == '+' and not operators[-1] == '*'))
                   ):
                if debug:
                    print('while there is an operator at the top of the'
                          + ' operator stack (not a left parenthesis): '
                          + operators[-1])
                    # and ((the operator at the top of the operator stack has
                    #  greater precedence) or (the operator at the top of the
                    #  operator stack has equal precedence and the token is
                    #  left associative))
                if debug:
                    print('pop operators from the operator stack onto the'
                          + f' output queue: {operators[-1]}')
                output.append(operators.pop())
            if debug:
                print(f'push it onto the operator stack: {token}')
            operators.append(token)
        elif token[0] == '(':
            if debug:
                print('else if the token is a left parenthesis then: '
                      + token[0])
            while token[0] == '(':
                if debug:
                    print(f'push it onto the operator stack: {token[0]}')
                operators.append(token[0])
                token = token[1:]
            if debug:
                print(f'push it to the output queue: {int(token)}')
            output.append(int(token))
        elif token[-1] == ')':
            rp_count = 0
            while token[-1] == ')':
                if debug:
                    print('else if the token is a right parenthesis: '
                          + token[-1])
                rp_count += 1
                token = token[:-1]
            if debug:
                print(f'push it to the output queue: {int(token)}')
            output.append(int(token))
            while rp_count > 0:
                rp_count -= 1
                # while the operator at the top of the operator stack is not
                # a left parenthesis:
                while operators[-1] != '(':
                    if debug:
                        print('pop operators from the operator stack onto'
                              + ' the output queue: {operators[-1]}')
                    output.append(operators.pop())
                    # /* If the stack runs out without finding a left
                    # parenthesis, then there are mismatched parentheses. */
                if debug:
                    print('if there is a left parenthesis at the top of the'
                          + 'operator stack, then: pop the operator from the'
                          + f'operator stack and discard it: {operators[-1]}')
                operators.pop()
        # else if the token is a number, then:
        else:
            if debug:
                print(f'push it to the output queue: {int(token)}')
            output.append(int(token))
    # /* After while loop, if operator stack not null,
    # pop everything to output queue */
    # if there are no more tokens to read then:
    #     while there are still operator tokens on the stack:
    while len(operators) > 0:
        # /* If the operator token on the top of the stack is a parenthesis,
        # then there are mismatched parentheses. */
        if debug:
            print('pop the operator from the operator stack onto the'
                  + f' output queue: {operators[-1]}')
        output.append(operators.pop())
    # exit.
    return output


def evaluate(expr_rpn):
    stack = []
    for token in expr_rpn:
        if token == '+':
            stack.append(stack.pop() + stack.pop())
        elif token == '*':
            stack.append(stack.pop() * stack.pop())
        else:
            stack.append(token)
    return stack.pop()

In [7]:
print(testdata[0])
expr_rpn = shunting_yard(testdata[0], debug=True)
print(expr_rpn)
evaluate(expr_rpn)

1 + 2 * 3 + 4 * 5 + 6
read a token: 1
push it to the output queue: 1
read a token: +
if token is an operator then: +
push it onto the operator stack: +
read a token: 2
push it to the output queue: 2
read a token: *
if token is an operator then: *
while there is an operator at the top of the operator stack (not a left parenthesis): +
pop operators from the operator stack onto the output queue: +
push it onto the operator stack: *
read a token: 3
push it to the output queue: 3
read a token: +
if token is an operator then: +
while there is an operator at the top of the operator stack (not a left parenthesis): *
pop operators from the operator stack onto the output queue: *
push it onto the operator stack: +
read a token: 4
push it to the output queue: 4
read a token: *
if token is an operator then: *
while there is an operator at the top of the operator stack (not a left parenthesis): +
pop operators from the operator stack onto the output queue: +
push it onto the operator stack: *
read 

71

In [8]:
for expr_str in testdata:
    expr_rpn = shunting_yard(expr_str)
    result = evaluate(expr_rpn)
    print(expr_str, expr_rpn, result)

1 + 2 * 3 + 4 * 5 + 6 [1, 2, '+', 3, '*', 4, '+', 5, '*', 6, '+'] 71
2 * 3 + (4 * 5) [2, 3, '*', 4, 5, '*', '+'] 26
5 + (8 * 3 + 9 + 3 * 4 * 3) [5, 8, 3, '*', 9, '+', 3, '+', 4, '*', 3, '*', '+'] 437
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) [5, 9, '*', 7, 3, '*', 3, '*', 9, '+', 3, '*', 8, 6, '+', 4, '*', '+', '*'] 12240
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 [2, 4, '+', 9, '*', 6, 9, '+', 8, '*', 6, '+', '*', 6, '+', 2, '+', 4, '+', 2, '*'] 13632


In [9]:
cumsum = 0
for expr_str in inputdata:
    expr_rpn = shunting_yard(expr_str)
    result = evaluate(expr_rpn)
    # print(expr_str, expr_rpn, result)
    cumsum += result
print(cumsum)

464478013511


In [10]:
HTML(downloaded['part1_footer'])

## Part Two

In [11]:
HTML(downloaded['part2'])

In [12]:
print(testdata[0])
expr_rpn = shunting_yard(testdata[0], part2=True, debug=True)
print(expr_rpn)
evaluate(expr_rpn)

1 + 2 * 3 + 4 * 5 + 6
read a token: 1
push it to the output queue: 1
read a token: +
if token is an operator then: +
push it onto the operator stack: +
read a token: 2
push it to the output queue: 2
read a token: *
if token is an operator then: *
while there is an operator at the top of the operator stack (not a left parenthesis): +
pop operators from the operator stack onto the output queue: +
push it onto the operator stack: *
read a token: 3
push it to the output queue: 3
read a token: +
if token is an operator then: +
push it onto the operator stack: +
read a token: 4
push it to the output queue: 4
read a token: *
if token is an operator then: *
while there is an operator at the top of the operator stack (not a left parenthesis): +
pop operators from the operator stack onto the output queue: +
push it onto the operator stack: *
read a token: 5
push it to the output queue: 5
read a token: +
if token is an operator then: +
push it onto the operator stack: +
read a token: 6
push it to

231

In [13]:
for expr_str in testdata:
    expr_rpn = shunting_yard(expr_str, part2=True)
    result = evaluate(expr_rpn)
    print(expr_str, expr_rpn, result)

1 + 2 * 3 + 4 * 5 + 6 [1, 2, '+', 3, 4, '+', 5, 6, '+', '*', '*'] 231
2 * 3 + (4 * 5) [2, 3, 4, 5, '*', '+', '*'] 46
5 + (8 * 3 + 9 + 3 * 4 * 3) [5, 8, 3, 9, 3, '+', '+', 4, 3, '*', '*', '*', '+'] 1445
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) [5, 9, 7, 3, 3, 9, '+', 3, 8, 6, '+', 4, '*', '+', '*', '*', '*', '*', '*'] 669060
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 [2, 4, '+', 9, '*', 6, 9, '+', 8, 6, '+', '*', 6, '+', '*', 2, 4, '+', '+', 2, '*'] 23340


In [14]:
cumsum = 0
for expr_str in inputdata:
    expr_rpn = shunting_yard(expr_str, part2=True)
    result = evaluate(expr_rpn)
    # print(expr_str, expr_rpn, result)
    cumsum += result
print(cumsum)

85660197232452


In [15]:
HTML(downloaded['part2_footer'])